In [3]:
import torch

from rqvae.rqvae_data import get_data

df = get_data()

In [2]:
embs = torch.stack(df["embeddings"].tolist())

In [3]:
import json
from utils import DEVICE
from models.base import BaseModel

config = json.load(open("../configs/train/tiger_train_config.json"))

rqvae_train_config = json.load(open(config['rqvae_train_config_path']))
rq_vae_config = rqvae_train_config['model']
rq_vae_config['should_init_codebooks'] = False

rqvae_model = BaseModel.create_from_config(rq_vae_config).to(DEVICE)

rqvae_model.load_state_dict(torch.load(config['rqvae_checkpoint_path'], weights_only=True))
rqvae_model.eval()

ids = df.asin_numeric.tolist()

embs_dict = {"ids": torch.tensor(ids).to(DEVICE), "embeddings": embs.to(DEVICE)}

semantic_ids = list(rqvae_model.forward(embs_dict))

In [4]:
from rqvae.collisions import dedup

items_with_tuples = list(zip(df["asin"], df["title"].fillna("unknown"), semantic_ids))
items_with_tuples = dedup(items_with_tuples)

assert len(df) == len(set(item[-1] for item in items_with_tuples))

In [ ]:
from rqvae.rqvae_data import search_similar_items


for i in range(5):
    sim = search_similar_items(items_with_tuples, (i,), 5)
    if len(sim) == 0:
        continue
    print(i)
    for asin, item, clust_tuple in sim:
        # if 'shampoo' in item.lower():
        print(f"{item=} {clust_tuple=}")

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt


plt.hist(Counter(item[-1][:-1] for item in items_with_tuples).values())
plt.show()

In [18]:
# # raw full ids
# full_duplicates = Counter(item[-1][:-1] for item in items_with_tuples).items()
# duplicated = [(semantic_id, amount) for (semantic_id, amount) in full_duplicates if amount > 1]
# duplicated

In [ ]:
# collison counters - (how many item have same full semantic id): amount of such sets
vals = Counter(item[-1][:-1] for item in items_with_tuples).values()
Counter(vals)

In [ ]:
# dedup idxes
Counter(item[-1][4] for item in items_with_tuples)

In [11]:
# from sklearn import preprocessing

# labels = df['asin']

# le = preprocessing.LabelEncoder()
# targets = le.fit_transform(labels)

# df['asin_numeric'] = targets

# torch.save(df, './all_data.pt')

In [ ]:
import torch
from utils import create_masked_tensor


embeddings = torch.rand((11, 2))
print(embeddings)

lengths = torch.tensor([3, 1, 2, 5])

padded_embeddings, mask = create_masked_tensor(embeddings, lengths)

In [ ]:
padded_embeddings

In [ ]:
mask

In [19]:
import pandas as pd

mapping = pd.read_csv("../data/Beauty/ratings_Beauty_full.csv")

In [20]:
deduped_mapping = mapping.drop_duplicates(subset=['item_id', 'raw_item_id'])

semantic_ids_extractor = {}
for idx, row in pd.merge(deduped_mapping, df, 'inner', left_on='raw_item_id', right_on='asin').iterrows():
    semantic_ids_extractor[row['item_id']] = row['embeddings']

In [32]:
torch.save(semantic_ids_extractor, "../data/Beauty/embs_extractor.pt")